# Make 01 matrix as hexagons heatmap

![Hexagons](./hexagons.png)

In [1]:
from bokeh.io import show
from bokeh.plotting import figure, ColumnDataSource
from bokeh.palettes import Inferno256
from bokeh.models import HoverTool, LinearColorMapper

TOOLS = "hover,save,pan,box_zoom,reset,wheel_zoom"

import numpy as np

### Base

In [2]:
q1 = np.pi*2/6

angles = np.arange(6)*q1
x = np.cos(angles)
y = np.sin(angles)

vx = 2
vy = 2 * np.sqrt(3/4)

### Random sample

In [3]:
mat = np.random.rand(5, 10)

In [4]:
r, c = mat.shape

### Size mapper

In [5]:
MAX = 500
w, h = None, None

if r > c :
    h = MAX
    w = int(MAX * c/r)
else:
    w = MAX
    h = int(MAX * r/c)

### Color mapper

In [6]:
def colormapper_01(v, palette=Inferno256):
    return Inferno256[int(v*256)]

## Figure maker

In [7]:
f = figure(width=w, height=h, title="Without hover", x_range=(-vx, vx*(c+0.5)), y_range=(-vy, vy*r))

f.background_fill_color = "black"
f.xgrid.grid_line_color = None
f.ygrid.grid_line_color = None

f.xaxis.minor_tick_line_color = None
f.yaxis.minor_tick_line_color = None
f.xaxis.major_tick_line_color = None
f.yaxis.major_tick_line_color = None

f.xaxis.major_label_text_font_size = '0pt'
f.yaxis.major_label_text_font_size = '0pt'

# row
for i in range(r):
    # cols
    for j in range(c):
        di = i
        
        f.patch(y + (j + (i%2)/2)*vx, x + i*vy, fill_color=colormapper_01(mat[i,j]), line_color=None)


show(f)

---

### With labels + `ColumnDataSource`

In [8]:
mat_label = np.arange(r*c).reshape((r, c)).astype(str) 

##### Prepare data

In [9]:
X = []
Y = []
V = []
L = []

for i in range(r):
    # cols
    for j in range(c):
        di = i
        X.append(y + (j + (i%2)/2)*vx)
        Y.append(x + i*vy)
        V.append(mat[i,j])
        L.append(mat_label[i,j])

##### Prepare figure

In [10]:
f = figure(width=w, height=h, title="With hover", x_range=(-vx, vx*(c+0.5)), y_range=(-vy, vy*r), tools=TOOLS)

f.background_fill_color = "black"
f.xgrid.grid_line_color = None
f.ygrid.grid_line_color = None

f.xaxis.minor_tick_line_color = None
f.yaxis.minor_tick_line_color = None
f.xaxis.major_tick_line_color = None
f.yaxis.major_tick_line_color = None

f.xaxis.major_label_text_font_size = '0pt'
f.yaxis.major_label_text_font_size = '0pt'


##### Prepare source

In [11]:
source = ColumnDataSource(data={"X":X, "Y":Y, "V":V, "L":L})
color_mapper = LinearColorMapper(palette="Inferno256", low=0, high=1)

##### Fill figure

In [12]:
f.patches("X", "Y",  
          source=source,
         fill_color={'field': 'V', 'transform': color_mapper},
         line_color=None)

f.select_one(HoverTool).tooltips = [
     ('Label', '@L'),
     ('Value', '@V'),
    ]


##### Display

In [13]:
show(f)